In [2]:
import pandas as pd
import requests
import time

[DIAMOND,EMERALD,PLATINUM,GOLD,SILVER,BRONZE,IRON]

In [65]:
import json
with open('server.json', 'r', encoding='utf-8') as fichier :
    donnees = json.load(fichier)

with open('api_key.json', 'r', encoding='utf-8') as fichier :
    api_key = json.load(fichier)["api_key"]
def fetch(json,match,feature,feature_chall,feature_teams):
    try :
        if json["info"]['queueId']==420 :
            data=[]
            for k in range(len(json['metadata']["participants"])) :
                
                data+=[json['metadata']["matchId"]]

                data+=[[json['info']["teams"][0]["bans"][k]["championId"] for k in range(5)] + [json['info']["teams"][1]["bans"][k]["championId"] for k in range(5)]]
                if k>5:
                    for feat in feature_teams :
                        data+=[json['info']["teams"][1][feat]]

                else : 
                    for feat in feature_teams :
                        data+=[json['info']["teams"][0][feat]]

                for feat_chall in feature_chall :
                    data+=[json['info']["participants"][k]["challenges"][feat_chall]]
                    

                for feat in feature :
                    data+=[json['info']["participants"][k][feat]]

                match+=[data]
                data=[]
    except Exception as e:
        print(f"Une erreur est survenue : {e}")
    return match

def getgame(elo,api_key): 

    headers = {
        "X-Riot-Token": api_key
    }
    region = 'euw1'
    request=0
    request_=0

    joueurs=[]
    for div in ["I","II","III","IV"] : 
        url = f"https://euw1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{elo}/{div}"
        for k in range(1,4):
            try :
                joueurs = requests.get(url+f"?page={k}", headers=headers).json()
                request+=1
            except Exception as e  :
                print(f"error : {e}")
                request+=1

    
    games=[]

    for joueur in joueurs :
        if request<99 :
            try :
                game=requests.get(f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{joueur["puuid"]}/ids?start=0&count=15',headers=headers).json()
                print(game)
                games+=game
                request+=1
            except Exception as e  :
                print(f"error : {e}")
                request+=1
        else :
            time.sleep(120)
            request=0  


    pd.DataFrame(columns=["gameId"],data=games).to_csv(f"gamesID_{elo}.csv", index=False)
    return games



def database(matches,donnees):
    USER = donnees["USER"]
    PASSWORD = donnees["PASSWORD"]
    HOST = "localhost"  # ou une adresse IP distante
    PORT = "5432"
    DB_NAME = "Champ_selectbdd"

    # Créer une connexion SQLAlchemy
    engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")
    try:
        # Essayer de se connecter
        with engine.connect() as connection:
            print("✅ Connexion réussie à la base de données via SQLAlchemy !")
    except Exception as e:
        print(f"❌ Erreur de connexion : {e}")

    matches.to_sql("Matchs", engine, if_exists="append", index=False)



def getmatchs(games,api_key,elo,donnees):
    headers = {
        "X-Riot-Token": api_key
    }

    region = 'euw1'
    feature=['puuid','championId',"deaths","kills","damageDealtToObjectives","damageDealtToBuildings","magicDamageDealt","magicDamageTaken","physicalDamageDealt","physicalDamageTaken",
             "timeCCingOthers","totalDamageShieldedOnTeammates","totalHeal","goldSpent","goldEarned","totalEnemyJungleMinionsKilled","totalAllyJungleMinionsKilled","lane","killingSprees","totalMinionsKilled"]
    
    feature_chall=["kda","killParticipation","goldPerMinute","damagePerMinute","soloKills","skillshotsDodged","skillshotsHit","visionScorePerMinute"]
    feature_teams=["teamId","win"]

    matchs=[]

    print(len(games))
    request=0

    for x in games:
        if request<99 :
            try :
                matchs= fetch(requests.get(f'https://europe.api.riotgames.com/lol/match/v5/matches/{x}',headers=headers).json(),matchs,feature,feature_chall,feature_teams)
                request+=1

            except Exception as e  :
                print(f"error : {e}")
        else :
            time.sleep(120)
            request=0  
            
    matchs=pd.DataFrame(data=matchs,columns=["matchId","bans"]+feature_teams+feature_chall+feature)
    matchs["elo"]=elo

    matchs.to_csv(f"matchesID_{elo}.csv", index=False)
    database(matchs,donnees)


In [ ]:
games = getgame("PLATINUM",api_key)

error
error
error


In [3]:
games = pd.read_csv("gamesID_PLATINUM.csv")["gameId"].values.tolist()

In [58]:
import os
elo = "DIAMOND"
file_path = f"gamesID_{elo}.csv"
os.path.exists(file_path) 

True

In [4]:
matches = getmatchs(games,api_key)

36882
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'killParticipation'
Une erreur est survenue : 'k

In [5]:
matches

,matchId,bans,teamId,win,kda,killParticipation,goldPerMinute,damagePerMinute,soloKills,skillshotsDodged,...,timeCCingOthers,totalDamageShieldedOnTeammates,totalHeal,goldSpent,goldEarned,totalEnemyJungleMinionsKilled,totalAllyJungleMinionsKilled,lane,killingSprees,totalMinionsKilled
0,EUW1_7307552867,"[81, 56, 83, 28, 800, 350, 40, 117, 82, 112]",100,False,1.285714,0.562500,366.490548,795.600731,1,15,...,68,0,6393,10400,10574,0,0,TOP,1,186
1,EUW1_7307552867,"[81, 56, 83, 28, 800, 350, 40, 117, 82, 112]",100,False,1.200000,0.375000,312.655111,156.457063,0,5,...,9,0,12508,9075,9021,0,0,JUNGLE,0,11
2,EUW1_7307552867,"[81, 56, 83, 28, 800, 350, 40, 117, 82, 112]",100,False,2.666667,0.500000,401.543168,681.093465,1,20,...,1,0,3958,9975,11586,0,0,TOP,1,239
3,EUW1_7307552867,"[81, 56, 83, 28, 800, 350, 40, 117, 82, 112]",100,False,1.800000,0.562500,376.775597,602.137184,0,13,...,16,0,3911,10600,10871,0,0,BOTTOM,2,212
4,EUW1_7307552867,"[81, 56, 83, 28, 800, 350, 40, 117, 82, 112]",100,False,3.666667,0.687500,252.225865,465.495884,0,15,...,16,3616,7929,6850,7277,0,0,BOTTOM,0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220370,EUW1_7269880697,"[51, 111, 19, 28, 711, 62, 555, 25, 82, -1]",100,True,0.666667,0.333333,322.576873,558.709962,0,20,...,25,0,1675,7275,7417,0,0,TOP,1,147
220371,EUW1_7269880697,"[51, 111, 19, 28, 711, 62, 555, 25, 82, -1]",200,False,0.750000,0.500000,339.571778,327.590238,2,53,...,102,0,10818,6800,7808,0,0,JUNGLE,1,34
220372,EUW1_7269880697,"[51, 111, 19, 28, 711, 62, 555, 25, 82, -1]",200,False,0.100000,0.083333,294.187581,585.198781,1,22,...,18,0,427,6800,6765,0,0,MIDDLE,0,155
220373,EUW1_7269880697,"[51, 111, 19, 28, 711, 62, 555, 25, 82, -1]",200,False,1.000000,0.500000,340.851550,463.911171,0,51,...,4,0,1720,7250,7838,0,0,BOTTOM,1,151


In [7]:
matches["elo"]="PLATINUM"

In [22]:
matches["matchId"].unique()

array(['EUW1_7307552867', 'EUW1_7307538859', 'EUW1_7307494734', ...,
       'EUW1_7270270093', 'EUW1_7270092138', 'EUW1_7269880697'],
      dtype=object)

In [15]:
matches.to_csv(f"matchesID_PLATINUM.csv", index=False)

In [40]:
for col in ["matchId","bans","puuid","lane","elo"]:  # Seulement les colonnes texte
    matches[col] = matches[col].astype(str)  # Convertir en string pour éviter les objets
    matches[col] = matches[col].apply(lambda x: x.encode("utf-8", "ignore").decode("utf-8") if isinstance(x, str) else x)

In [61]:
from sqlalchemy import create_engine
import json
with open('server.json', 'r', encoding='utf-8') as fichier :
    donnees = json.load(fichier)
# ⚡ Connexion à PostgreSQL
USER = donnees["USER"]
PASSWORD = donnees["PASSWORD"]
HOST = "localhost"  # ou une adresse IP distante
PORT = "5432"
DB_NAME = "Champ_selectbdd"

# Créer une connexion SQLAlchemy
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")
try:
    # Essayer de se connecter
    with engine.connect() as connection:
        print("✅ Connexion réussie à la base de données via SQLAlchemy !")
except Exception as e:
    print(f"❌ Erreur de connexion : {e}")
# ⚡ Insérer le DataFrame dans la table "ma_table"

matches.to_sql("Matchs", engine, if_exists="append", index=False)

✅ Connexion réussie à la base de données via SQLAlchemy !


595

In [38]:
matches["matchId"].astype(str)

0         EUW1_7307552867
1         EUW1_7307552867
2         EUW1_7307552867
3         EUW1_7307552867
4         EUW1_7307552867
               ...       
220370    EUW1_7269880697
220371    EUW1_7269880697
220372    EUW1_7269880697
220373    EUW1_7269880697
220374    EUW1_7269880697
Name: matchId, Length: 220375, dtype: object

In [36]:
print(matches.dtypes)


matchId                            object
bans                               object
teamId                              int64
win                                  bool
kda                               float64
killParticipation                 float64
goldPerMinute                     float64
damagePerMinute                   float64
soloKills                           int64
skillshotsDodged                    int64
skillshotsHit                       int64
visionScorePerMinute              float64
puuid                              object
championId                          int64
deaths                              int64
kills                               int64
damageDealtToObjectives             int64
damageDealtToBuildings              int64
magicDamageDealt                    int64
magicDamageTaken                    int64
physicalDamageDealt                 int64
physicalDamageTaken                 int64
timeCCingOthers                     int64
totalDamageShieldedOnTeammates    

In [6]:
url = f"https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7307552867"
headers = {
        "X-Riot-Token": 'RGAPI-77af1994-62c2-45d5-b935-6c4599916ac3'
    }                               

requests.get(url,headers=headers).json()

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_7307552867',
  'participants': ['FP2mPOsn98JL2pc6-Vcq7dLXd8xzkfueneGAg1CPR0ulgZR6r75U4DiD3VT9vWFVll2AS7-OpWbODg',
   'rjy4Fb871VwXAJ2aCpMbWTrlbwYUo8RDgoY_I2336l9-9kL81qkwfTve-z-fjsoCyHvOXMUFBb_nug',
   'hOFTuHsLHRmdUBhJYGmcs9jPdGCgPXeOyW1yPaZGkjGZow94ZlIx317n5hJykLtArE4Z9Hb_E2DlPg',
   'V7XZAjrLTl5P7r_8_bTWnHtyqw8Aba69-Q42mFvRaqk-RNkPB-647h6LY48-qqBq4HB47nyEfRTHBQ',
   '8S8yQO4J_jZ1Z6_wj8ae4g8v6dkXvf3G3APDVk6EINJO-SPY0i6chySHW7_0pjlVtPIib8htGPJXZQ',
   'NXZFoexfen2sLx883xnT0Q0EdeKJ3La1awcoharZAxLvMISIN2C7hdGDCv8Lsj7WVuIrgfAXLDSiTA',
   'k24CXys5SQrTI4Nt5pKUppIu1-ReIPyWjfsewGUXUho87DgF1CSBGXm5a6pb6oswYJR2T_h80CJgFg',
   'Mc4WYffFf0Y9KzFXn6gW6pu0UcaNBrT7-uX6MhqmqTePz9Vecwnp-IGq5Bm9L4UdbheZfW4uMyUTqA',
   'Ttj62qc40e3250PA2puuegCNjwe0XX8-PVFGTsS6BtMlJxvvoWoLhqPTOG4mOqJi3FCUvx-V7myyoA',
   'Zk1SeucH3mnyPylQ_ne6zyDppSGsO5Dv2xEqhLPXS2S-xQD-MDCI5uZRGgmwjsPnl_jYyJzqztrchg']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 173

In [ ]:
url = f"https://europe.api.riotgames.com//lol/rso-match/v1/matches/jkoISeqpqCy5cr_NnNcuDjWKuXAQp5VmaoPxYwYRAwNuBHzivMXqMAzq61bNdKcutjzSyYTGBfr2XQ"
headers = {
        "X-Riot-Token": 'RGAPI-421ff416-7b0f-4afd-b109-752e7f7f2fb7'
    }
requests.get(url,headers=headers).json()

{'status': {'message': 'Forbidden', 'status_code': 403}}